# GLGENN on simple CV tasks

In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import os
os.chdir('/')
if not os.path.exists("/clifford-group-equivariant-neural-networks"):
    !git clone https://github.com/DavidRuhe/clifford-group-equivariant-neural-networks.git
os.chdir("/clifford-group-equivariant-neural-networks")

In [3]:
import sys
module_path = os.path.abspath('/kaggle/input')

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets
import torchvision.transforms.v2 as v2

from glgennc.glgenn.algebra.cliffordalgebraex import CliffordAlgebraQT
from glgennc.glgenn.algebra.cliffordalgebra import CliffordAlgebra
from glgennc.glgenn.layers.qtgp import QTGeometricProduct
from glgennc.glgenn.layers.qtlinear import QTLinear
from glgennc.glgenn.layers.qtnorm import QTNormalization

import matplotlib.pyplot as plt

In [5]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

## Dataloader (Colored Fashion MNIST / CIFAR-10)


In [6]:
class CliffordFashionedMnist(Dataset):
    def __init__(self, root, train=True, download=True, d=3):
        self.metric = [1] * d
        self.ca = CliffordAlgebraQT(self.metric)
        self.d = d
        self.post_transforms = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.data = datasets.FashionMNIST(root, train=train, download=download)
        _, self.h, self.w = self.data.data.shape
        # y_coords, x_coords = torch.meshgrid(
        #     torch.linspace(-1, 1, self.h),
        #     torch.linspace(-1, 1, self.w),
        #     indexing="ij"
        # )
        # self.grid = torch.stack([x_coords, y_coords], dim=0) # [2, 28, 28]

    def _colorize_random(self, img):
        img_np = np.array(img)
        factors = np.random.uniform(0.2, 1.0, 3)
        color_img = np.stack([img_np * f for f in factors], axis=-1).astype(np.uint8) # [28, 28, 3]
        return color_img

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_raw, label = self.data[idx]
        img_colored = self._colorize_random(img_raw) # [3, 28, 28]
        img_tensor = self.post_transforms(img_colored)
        v_full = img_tensor #torch.cat([self.grid, img_tensor], dim=0) # [3, 28, 28]
        v_full = v_full.permute(1, 2, 0) # [28, 28, 3]
        return v_full, label

In [7]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import v2

class CliffordCIFAR10(Dataset):
    def __init__(self, root, train=True, download=True, d=3):
        self.metric = [1] * d
        self.ca = CliffordAlgebraQT(self.metric)
        self.d = d
        self.post_transforms = v2.Compose([
            v2.ToImage(),
            v2.ToDtype(torch.float32, scale=True),
            v2.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.data = datasets.CIFAR10(root, train=train, download=download)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_raw, label = self.data[idx]
        img_tensor = self.post_transforms(img_raw) # [3, 32, 32]
        v_full = img_tensor.permute(1, 2, 0) 
        return v_full, label

### Prepare: MVSiLU from CGENN

In [8]:
def unsqueeze_like(tensor: torch.Tensor, like: torch.Tensor, dim=0):
    """
    Unsqueeze last dimensions of tensor to match another tensor's number of dimensions.

    Args:
        tensor (torch.Tensor): tensor to unsqueeze
        like (torch.Tensor): tensor whose dimensions to match
        dim: int: starting dim, default: 0.
    """
    n_unsqueezes = like.ndim - tensor.ndim
    if n_unsqueezes < 0:
        raise ValueError(f"tensor.ndim={tensor.ndim} > like.ndim={like.ndim}")
    elif n_unsqueezes == 0:
        return tensor
    else:
        return tensor[dim * (slice(None),) + (None,) * n_unsqueezes]

class MVSiLU(nn.Module):
    def __init__(self, algebra, channels, invariant="mag2", exclude_dual=False):
        super().__init__()
        self.algebra = algebra
        self.channels = channels
        self.exclude_dual = exclude_dual
        self.invariant = invariant
        self.a = nn.Parameter(torch.ones(1, channels, algebra.dim + 1))
        self.b = nn.Parameter(torch.zeros(1, channels, algebra.dim + 1))

        if invariant == "norm":
            self._get_invariants = self._norms_except_scalar
        elif invariant == "mag2":
            self._get_invariants = self._mag2s_except_scalar
        else:
            raise ValueError(f"Invariant {invariant} not recognized.")

    def _norms_except_scalar(self, input):
        return self.algebra.norms(input, grades=self.algebra.grades[1:])

    def _mag2s_except_scalar(self, input):
        return self.algebra.qs(input, grades=self.algebra.grades[1:])

    def forward(self, input):
        norms = self._get_invariants(input)
        norms = torch.cat([input[..., :1], *norms], dim=-1)
        a = unsqueeze_like(self.a, norms, dim=2)
        b = unsqueeze_like(self.b, norms, dim=2)
        norms = a * norms + b
        norms = norms.repeat_interleave(self.algebra.subspaces, dim=-1)
        return torch.sigmoid(norms) * input

In [9]:
class CGEBlock(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()
        print(in_features)

        self.layers = nn.Sequential(
                QTGeometricProduct(algebra, out_features),
                QTNormalization(algebra, out_features),
            )
    def forward(self, input):
        # [batch_size, in_features, 2**d] -> [batch_size, out_features, 2**d]
        return self.layers(input)

In [10]:
class CGEMLP(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_layers=2):
        super().__init__()

        layers = []
        for i in range(n_layers - 1):
            layers.append(
                CGEBlock(algebra, in_features, hidden_features)
            )
            in_features = hidden_features
        layers.append(
            CGEBlock(algebra, hidden_features, out_features)
        )
        self.layers = nn.Sequential(*layers)

    def forward(self, input):
        return self.layers(input)

## Model: GLGENN

In [11]:
import torch
from torch import nn

class CliffordFashionModel(nn.Module):
    def __init__(self, ca, in_channels, hidden_channels, out_classes):
        super().__init__()
        self.ca = ca
        self.hidden_channels = hidden_channels
        self.cge_part = CGEMLP(ca, in_channels, hidden_channels, hidden_channels, 2)
        self.final_norm = QTNormalization(ca, hidden_channels)
        self.clifford_head = nn.Sequential(
            QTLinear(ca, in_channels, out_classes)
        )

    def forward(self, x):
        x = self.ca.embed_grade(x, 1)
        batch_size, h, w, mv_dim = x.shape
        x_processed = x.view(batch_size, h*w, mv_dim)
        h_out = self.cge_part(x_processed)
        flat_mvs = h_out.reshape(batch_size, h * w, mv_dim)
        logits_mv = self.clifford_head(flat_mvs) 
        return logits_mv[..., 0]

## Training & Evaluating

In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision.transforms.functional as TF
from tqdm import tqdm
import time
import math

def train(model, train_loader, test_loader, optimizer, criterion, epochs, device):
    for epoch in range(epochs):
        model.train()
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        correct = 0
        total = 0
        grad_norms = []  
        
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()

            # logging gradient norms
            total_norm = 0.0
            for p in model.parameters():
                if p.grad is not None:
                    param_norm = p.grad.detach().data.norm(2)
                    total_norm += param_norm.item() ** 2
            total_norm = total_norm ** 0.5
            grad_norms.append(total_norm)
 
            optimizer.step()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            avg_grad = sum(grad_norms[-10:]) / len(grad_norms[-10:])
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}', 
                'acc': f'{100.*correct/total:.2f}%',
                'grad': f'{avg_grad:.2f}'
            })
        model.eval()
        test_correct = 0
        rot_correct = 0
        test_total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = outputs.max(1)
                test_correct += predicted.eq(labels).sum().item()
                rotated_images = TF.rotate(images, angle=180) 
                outputs_rot = model(rotated_images)
                _, predicted_rot = outputs_rot.max(1)
                rot_correct += predicted_rot.eq(labels).sum().item()
                test_total += labels.size(0)
        
        test_acc = 100. * test_correct / test_total
        rot_acc = 100. * rot_correct / test_total
        
        print(f'Epoch {epoch+1}, Inital Accuracy: {test_acc:.2f}%')
        print(f'Epoch {epoch+1}, Rotated Accuracy: {rot_acc:.2f}%')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

d = 3
lr = 0.001
epochs = 10
batch_size = 256

ca = CliffordAlgebraQT([1] * d)
full_dataset = CliffordCIFAR10(root='./data', train=True, download=True, d=d)
test_dataset = CliffordCIFAR10(root='./data', train=False, download=True, d=d)

train_loader = DataLoader(
    full_dataset, 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=4, 
    pin_memory=True          
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=4, 
    pin_memory=True          
)

model = CliffordFashionModel(ca, in_channels=1024, hidden_channels=1024, out_classes=10).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

train(model, train_loader, test_loader, optimizer, criterion, epochs, device)

1024
1024


Epoch 1/10: 100%|██████████| 196/196 [00:30<00:00,  6.47it/s, loss=1.7568, acc=35.84%, grad=1.01]


Epoch 1, Inital Accuracy: 42.27%
Epoch 1, Rotated Accuracy: 41.18%


Epoch 2/10: 100%|██████████| 196/196 [00:29<00:00,  6.62it/s, loss=1.2598, acc=50.12%, grad=1.10]


Epoch 2, Inital Accuracy: 45.42%
Epoch 2, Rotated Accuracy: 43.26%


...

Epoch 8, Inital Accuracy: 46.98%
Epoch 8, Rotated Accuracy: 44.97%


Epoch 9/10: 100%|██████████| 196/196 [00:29<00:00,  6.63it/s, loss=0.1418, acc=96.04%, grad=0.76]


Epoch 9, Inital Accuracy: 47.18%
Epoch 9, Rotated Accuracy: 45.18%


Epoch 10/10: 100%|██████████| 196/196 [00:29<00:00,  6.64it/s, loss=0.2109, acc=97.13%, grad=0.82]


Epoch 10, Inital Accuracy: 47.34%
Epoch 10, Rotated Accuracy: 46.05%


## Model: just MLP

In [13]:
class SimpleMLP(nn.Module):
    def __init__(self, in_channels=1, hidden_dim=128, out_classes=10):
        super().__init__()
        self.flatten = nn.Flatten()
        
        self.model = nn.Sequential(
            nn.Linear(1024 * in_channels, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, out_classes)
        )
    def forward(self, x):
        out = self.flatten(x)
        out = self.model(out)
        return out

model = SimpleMLP(in_channels=3, hidden_dim=128).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train(model, train_loader, test_loader, optimizer, criterion, epochs, device)

Epoch 1/10: 100%|██████████| 196/196 [00:07<00:00, 27.95it/s, loss=1.6468, acc=41.70%, grad=1.54]


Epoch 1, Inital Accuracy: 46.32%
Epoch 1, Rotated Accuracy: 22.13%


Epoch 2/10: 100%|██████████| 196/196 [00:06<00:00, 28.16it/s, loss=1.4123, acc=48.82%, grad=1.55]


Epoch 2, Inital Accuracy: 49.16%
Epoch 2, Rotated Accuracy: 23.38%


Epoch 3/10: 100%|██████████| 196/196 [00:06<00:00, 28.75it/s, loss=1.1526, acc=52.04%, grad=1.61]


...

Epoch 8, Inital Accuracy: 51.76%
Epoch 8, Rotated Accuracy: 25.70%


Epoch 9/10: 100%|██████████| 196/196 [00:06<00:00, 28.17it/s, loss=1.0408, acc=61.11%, grad=1.77]


Epoch 9, Inital Accuracy: 51.66%
Epoch 9, Rotated Accuracy: 24.15%


Epoch 10/10: 100%|██████████| 196/196 [00:06<00:00, 28.33it/s, loss=0.9542, acc=62.11%, grad=1.90]


Epoch 10, Inital Accuracy: 52.11%
Epoch 10, Rotated Accuracy: 24.36%
